In [1]:
import lightgbm as lgb
import numpy as np
import pandas as pd
import numpy as np
import gc
import os
import optuna
import sqlite3
import ray
import matplotlib.pyplot as plt
from optuna.integration import LightGBMPruningCallback
from autogluon.tabular import TabularPredictor
from sklearn.metrics import mean_squared_error, mean_absolute_error

In [2]:
gc.collect()
df_full = pd.read_parquet('./data/l_vm_completa_normalizada_fe.parquet', engine='fastparquet')

In [3]:
# Variables predictoras y objetivo
# filtrar que en X el periodo sea menor o igual a 201910
# En x eliminar la columna 'CLASE_ZSCORE' y 'CLASE_DELTA_ZSCORE' si existen
cols_to_drop = [col for col in ['CLASE_ZSCORE', 'CLASE_DELTA_ZSCORE','CLASE_ZSCORE_CLIP', 'CLASE_DELTA_ZSCORE_CLIP'] if col in df_full.columns]
X = df_full[df_full['PERIODO'] <= 201910].drop(columns=cols_to_drop)
# Filtrar en y que el periodo sea menor o igual a 201910 y que la columna exista
y = df_full[df_full['PERIODO'] <= 201910]['CLASE_DELTA_ZSCORE_CLIP']
# Eliminar df_full para liberar memoria
del df_full
gc.collect()

48

In [4]:
# Definir los periodos de validación 
periodos_valid = [201909, 201910]

# Separar train y cinco conjuntos de validación respetando la secuencia temporal
X_train = X[X['PERIODO'] < periodos_valid[0]]
y_train = y[X['PERIODO'] < periodos_valid[0]]
X_val_list = [X[X['PERIODO'] == p] for p in periodos_valid]
y_val_list = [y[X['PERIODO'] == p] for p in periodos_valid]
del X, y
gc.collect()

0

In [5]:
from autogluon.tabular import TabularPredictor

# Dataset de entrenamiento
df_train = X_train.copy()
df_train['CLASE_DELTA_ZSCORE_CLIP'] = y_train

# Dataset de validación
df_val = X_val_list[0].copy()
df_val['CLASE_DELTA_ZSCORE_CLIP'] = y_val_list[0]

In [6]:
""" import datetime
import gc
from autogluon.tabular import TabularPredictor

gc.collect()

# === Configuración general ===
label_column = 'CLASE_DELTA_ZSCORE_CLIP'
output_path = f"AutogluonModels/model_fast_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}"

# === Entrenamiento ===
predictor = TabularPredictor(
    label=label_column,
    problem_type='regression',
    eval_metric='mean_absolute_error',
    path=output_path
).fit(
    train_data=df_train,
    tuning_data=df_val,
    use_bag_holdout=True,
    presets='medium_quality',               # Entrenamiento rápido pero razonable
    time_limit=1800,                         # ⏱️ Máximo 15 minutos (900 segundos)
    num_bag_folds=3,                        # Reduce cantidad de folds
    num_bag_sets=1,                         # Una sola pasada
    auto_stack=False,                       # Apagar stacking para acelerar
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    hyperparameters={
        'GBM': [
            {
                'ag_args': {'name_suffix': 'QuickXT'},
                'extra_trees': True,
                'time_limit': 300,
                'early_stopping_rounds': 10,
            },
            {
                'ag_args': {'name_suffix': 'QuickDefault'},
                'num_leaves': 31,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'time_limit': 300,
                'early_stopping_rounds': 10,
            }
        ]
    },
    verbosity=2
)
 """

' import datetime\nimport gc\nfrom autogluon.tabular import TabularPredictor\n\ngc.collect()\n\n# === Configuración general ===\nlabel_column = \'CLASE_DELTA_ZSCORE_CLIP\'\noutput_path = f"AutogluonModels/model_fast_{datetime.datetime.now().strftime(\'%Y%m%d_%H%M\')}"\n\n# === Entrenamiento ===\npredictor = TabularPredictor(\n    label=label_column,\n    problem_type=\'regression\',\n    eval_metric=\'mean_absolute_error\',\n    path=output_path\n).fit(\n    train_data=df_train,\n    tuning_data=df_val,\n    use_bag_holdout=True,\n    presets=\'medium_quality\',               # Entrenamiento rápido pero razonable\n    time_limit=1800,                         # ⏱️ Máximo 15 minutos (900 segundos)\n    num_bag_folds=3,                        # Reduce cantidad de folds\n    num_bag_sets=1,                         # Una sola pasada\n    auto_stack=False,                       # Apagar stacking para acelerar\n    ag_args_ensemble={\'fold_fitting_strategy\': \'parallel_local\'},\n    hyperpa

In [7]:
predictor = TabularPredictor.load("AutogluonModels/model_fast_20250629_0112")


In [ ]:
predictor.fit_extra(
    time_limit=1800,  # 30 minutos
    ag_args_ensemble={
        'fold_fitting_strategy': 'sequential_local',
        'num_bag_folds': 1  # 👈 solo 1 fold para asegurar que termine
    },
    fit_weighted_ensemble=True,
    hyperparameters={
        'NN_TORCH': [{
            'num_epochs': 4,
            'learning_rate': 0.01,
            'dropout_prob': 0.1,
            'weight_decay': 1e-5,
            'batch_size': 256,
            'hidden_size': 128,
            'ag_args': {'name_suffix': 'NN_LightFast'},
            'ag_args_fit': {
                'num_gpus': 1,  # Usa GPU si está disponible
            }
        }]
    },
    verbosity=2
)

Fitting 1 L1 models, fit_strategy="sequential" ...
Fitting model: NeuralNetTorchNN_LightFast_BAG_L1 ... Training model for up to 1800.00s of the 1800.00s of remaining time.
	Fitting 3 child models (S1F1 - S1F3) | Fitting with SequentialLocalFoldFittingStrategy


In [ ]:
predictor.fit_extra(
    time_limit=600,  # 5 minutos
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    fit_weighted_ensemble=True,
    hyperparameters={
        'CAT': [{'ag_args': {'name_suffix': 'CAT_Quick'}
        }]
    },
    verbosity=2
)

In [ ]:
predictor.fit_extra(
    time_limit=600,  # 5 minutos
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    fit_weighted_ensemble=True,
    hyperparameters={
        'XGB': [{
            'n_estimators': 150,
            'early_stopping_rounds': 10,
            'learning_rate': 0.05,
            'booster': 'gbtree',
            'ag_args': {'name_suffix': 'XGB_Quick'}
        }]
    },
    verbosity=2
)

In [ ]:
import datetime
gc.collect()
# === Configuración general ===
label_column = 'CLASE_DELTA_ZSCORE_CLIP'
output_path = f"AutogluonModels/model_gbm_{datetime.datetime.now().strftime('%Y%m%d_%H%M')}"

# === Entrenamiento ===
predictor = TabularPredictor(
    label=label_column,
    problem_type='regression',
    eval_metric='mean_absolute_error',
    path=output_path
).fit(
    train_data=df_train,
    tuning_data=df_val,
    use_bag_holdout=True,
    presets='medium_quality',
    num_bag_folds=8,         # Mayor robustez
    num_bag_sets=2,          # Mejor estimación out-of-fold
    auto_stack=True,         # Activar stacking desde el inicio
    time_limit=10800,
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    hyperparameters={
        'GBM': [
            {
                'ag_args': {'name_suffix': 'XT'},
                'extra_trees': True,
                'time_limit': 600,
                'early_stopping_rounds': 20,
            },
            {
                'ag_args': {'name_suffix': 'LinearTree'},
                'linear_tree': True,
                'time_limit': 600,
                'early_stopping_rounds': 20,
            },
            {
                'ag_args': {'name_suffix': 'Default'},
                'num_leaves': 31,
                'feature_fraction': 0.8,
                'bagging_fraction': 0.8,
                'time_limit': 600,
                'early_stopping_rounds': 20,
            }
        ]
    }
)


In [ ]:
# === Cargar modelo ya entrenado ===
#predictor = TabularPredictor.load("AutogluonModels/model_gbm")

predictor.fit_extra(
    time_limit=10800,  # Tiempo total disponible para todos los modelos nuevos
    ag_args_ensemble={'fold_fitting_strategy': 'parallel_local'},
    fit_weighted_ensemble=True,  # Activar Weighted Ensemble
    hyperparameters={
        'XGB': [{
            'n_estimators': 1000,
            'early_stopping_rounds': 25,
            'learning_rate': 0.03,
            'booster': 'gbtree',
            'ag_args': {'name_suffix': 'XGB_Custom'}
        }],
        'CAT': [{
            'ag_args_fit': {'time_limit': 1200},
            'ag_args': {'name_suffix': 'CAT_Custom'}
        }],
        'NN_TORCH': [{
            'num_epochs': 50,
            'learning_rate': 0.003,
            'layers': [1024, 512, 256],
            'dropout_prob': 0.3,
            'weight_decay': 1e-5,
            'batch_size': 2048,
            'ag_args': {'name_suffix': 'DeepCustom'},
            'ag_args_fit': {'num_gpus': 1}
        }]
    },
    verbosity=2
)


In [ ]:
# Comparar MAE y MedAE en el leaderboard
lb = predictor.leaderboard(data = df_val, extra_metrics=['mean_absolute_error', 'median_absolute_error', 'r2'], silent=False)

In [ ]:
predictor.get_model_best().get_info()['stacker_model'].weights

In [ ]:
# Evaluar feature importance sobre el conjunto de validación
importancia = predictor.feature_importance(df_val)

In [ ]:
# Combinar entrenamiento + validación
df_full = pd.concat([df_train, df_val], axis=0)
del df_train, df_val
gc.collect()

In [ ]:
# Reentrenar el mejor modelo con TODOS los datos disponibles
predictor_full = predictor.refit_full(train_data=df_full)

In [ ]:
# Verificar los modelos disponibles (el mejor ahora tiene el sufijo '_FULL')
print("Modelos disponibles luego del refit completo:")
print(predictor.leaderboard(silent=True)['model'].tolist())
# Eliminar modelos intermedios para liberar espacio
predictor.delete_models(models_to_keep='best', dry_run=False)

# Confirmar que solo queda el modelo reentrenado
print("\nModelos restantes después de eliminar los intermedios:")
print(predictor.leaderboard(silent=True)['model'].tolist())

# (Opcional) Guardar el predictor final si querés usarlo luego sin volver a cargar todo
predictor.save('./data/modelo_final_autogluon')

# ---  Liberar memoria ---
del df_full
gc.collect()


In [ ]:
# Cargo los datos sobre los que quiero hacer predicciones
df_pred_full = pd.read_parquet('./data/l_vm_completa_normalizada_fe.parquet', engine='fastparquet')
# Dejo solo los datos del periodo 201910 y que A_PREDECIR sea True
# Filtrar solo los datos del periodo 201910 y donde A_PREDECIR sea True
df_pred_full = df_pred_full[
    (df_pred_full['PERIODO'] == 201910) & (df_pred_full['A_PREDECIR'] == True)
].drop(columns=['CLASE_ZSCORE', 'CLASE_DELTA_ZSCORE'])

In [ ]:
# Realizar las predicciones usando el predictor original
predictions = predictor.predict(df_pred_full)
# Agregar las predicciones al DataFrame original
df_pred_full['CLASE_DELTA_ZSCORE'] = predictions

In [ ]:
# Imprimir la lista de columas del DataFrame con las predicciones
print("Columnas del DataFrame con las predicciones:")
print(df_pred_full.columns.tolist())

In [ ]:
# Dernormalizar la columna CLASE_DELTA_ZSCORE
df_pred_full['CLASE_DELTA'] = df_pred_full['CLASE_DELTA_ZSCORE'] * df_pred_full['CLASE_DELTA_STD'] + df_pred_full['CLASE_DELTA_MEAN']
df_pred_full['TN'] = df_pred_full['TN_ZSCORE'] * df_pred_full['TN_STD'] + df_pred_full['TN_MEAN']
# Agregar la columna TN_PREDICT que sea la suma de TN y CLASE_DELTA y si es menor que cero, poner cero
df_pred_full['TN_PREDICT'] = df_pred_full['TN'] + df_pred_full['CLASE_DELTA']
df_pred_full['TN_PREDICT'] = df_pred_full['TN_PREDICT'].clip(lower=0)

In [ ]:
# Generar Dataframe que contenga por cada PRODUCT_ID la suma de TN_PREDICT
df_final = df_pred_full.groupby('PRODUCT_ID').agg({'TN_PREDICT': 'sum'}).reset_index()
df_final = df_final.rename(columns={'PRODUCT_ID': 'product_id', 'TN_PREDICT': 'tn'})
# Guardar el DataFrame df_final en un archivo CSV
df_final.to_csv('./modelos/autoglun_normalizando_clase_delta.csv', index=False)
df_final.shape
